<a href="https://colab.research.google.com/github/ganeshred/NLPAuthorshipAttributionNLM/blob/main/sentence_pool_embeddings_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [5]:
import pandas as pd

In [6]:
# df = pd.read_csv('/content/gdrive/MyDrive/Task3.csv')

In [7]:
ctrl_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_ctrl.csv')
fair_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_fair.csv')
gpt_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_gpt.csv')
gpt2_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_gpt2.csv')
grover_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_grover.csv')
human_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_human.csv')
pplm_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_pplm.csv')
xlm_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_xlm.csv')
xlnet_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_xlnet.csv')
instruct_df = pd.read_csv('/content/gdrive/MyDrive/NLP/instruct.csv')
gpt3_df = pd.read_csv('/content/gdrive/MyDrive/NLP/gpt3.csv')

In [8]:
ctrl_df = ctrl_df[['Generation','label']]
fair_df = fair_df[['Generation','label']]
gpt_df = gpt_df[['Generation','label']]
gpt3_df = gpt3_df[['Generation','label']]
gpt2_df = gpt2_df[['Generation','label']]
grover_df = grover_df[['Generation','label']]
human_df = human_df[['Generation','label']]
pplm_df = pplm_df[['Generation','label']]
xlm_df = xlm_df[['Generation','label']]
xlnet_df = xlnet_df[['Generation','label']]
instruct_df = instruct_df[['Generation','label']]


In [9]:
ctrl_df['Generation'][0]

'latest headlines on cnn business tl;dr the u.s. government is expected to announce a new round of sanctions against iran next week, which could include the freezing or closing down iranian banks and other financial institutions, according to... [cartoon] -- a senior white house official said monday that president barack obama will sign an executive order this week imposing additional economic penalties aimed at pressuring tehran over its nuclear program. in addition, secretary of state hillary rodham clinton plans t...more from moneywatch ". | report as abusive, click here first published in money & business, march 26, 2009 8:28 a.m. et ============================================================ stocks tumble after report shows us economy shrinking for first time since 2008 - wall street journal http://on.wsj.com/1o6p2lh markets were lower monday, with dow jones industrial average falling more than 200 points, while treasury yields fell sharply. see full story. for international mark

In [10]:
import spacy
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string


nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# def get_sentence_embeddings(article):
#     doc = nlp(article)
#     sentence_embeddings = []
#     for sent in doc.sents:
#         # print(sent)
#         # Compute the average GloVe embedding for each word in the sentence
#         embeddings = []
#         for word in sent:
#           if not word.is_punct and not word.is_stop:
#             print(nlp.vocab.get_vector(word.text.lower()))


#     #     embeddings = [nlp.vocab.get_vector(word.text.lower()) for word in sent if not word.is_punct and not word.is_stop]
#     #     print(embeddings)
#     #     if embeddings:
#     #         sentence_embeddings.append(np.mean(embeddings, axis=0))
#     # return sentence_embeddings


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
import torch

# Load GloVe embeddings
embeddings = {}
with open("/content/gdrive/MyDrive/NLP/glove.6B.50d.txt", "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = torch.tensor([float(val) for val in values[1:]])
        embeddings[word] = vector

def get_sentence_embeddings(article):
    doc = nlp(str(article))
    sentence_embeddings = []
    sents = list(doc.sents)
    for i in range(0,len(list(sents)),5):
    # for sent in doc.sents:
        # Compute the average GloVe embedding for each word in the sentence
        if (i+4 <= len(list(doc.sents))-1):
          super_sent = list(doc.sents)[i:i+5]
          sent = ''.join(str(super_sent))
          sent = nlp(sent)
        else:
          super_sent = list(doc.sents)[i]
          sent = ''.join(str(super_sent))
          sent = nlp(sent)
        sent_embeddings = []
        for word in sent:
            if not word.is_punct and not word.is_stop:
                embedding = embeddings.get(word.lower_, None)
                if embedding is not None and embedding != []:
                    sent_embeddings.append(embedding)
        if len(sent_embeddings) > 0:
            sentence_embedding = torch.stack(sent_embeddings).mean(dim=0)
            sentence_embeddings.append(sentence_embedding)
    return sentence_embeddings


In [12]:
get_sentence_embeddings('')

[]

In [13]:
# combined_df1['label'] = combined_df1['label'].str.lower()

In [14]:
# combined_df.head()

In [15]:
combined_df = pd.concat([ctrl_df,fair_df,gpt_df,gpt3_df,gpt2_df,grover_df,human_df,pplm_df,xlm_df,xlnet_df,instruct_df], ignore_index=True)


In [16]:
combined_df[['Generation']] = combined_df[['Generation']].astype(str)

In [17]:
# combined_df['GenMatrix'] = combined_df[['Generation']].apply(get_sentence_embeddings)
# combined_df['GenMatrix'] = combined_df.apply(lambda row: get_sentence_embeddings(row['Generation']), axis=1)


In [18]:
# combined_df['GenMatrix']
# torch.save(combined_df['GenMatrix'], '/content/gdrive/MyDrive/NLP/gen_embeddings.pt')

In [19]:
combined_df['label'] = combined_df['label'].str.lower()

In [20]:
combined_df['GenMatrix'] = torch.load('/content/gdrive/MyDrive/NLP/gen_embeddings.pt')

In [21]:
embeddings = torch.load('/content/gdrive/MyDrive/NLP/gen_embeddings.pt')

In [22]:
embeddings[1]

[tensor([ 5.5902e-01, -1.5989e-01,  2.1612e-01, -2.5526e-02,  1.4165e-01,
          5.4916e-02, -3.2942e-01,  2.3700e-03,  4.9956e-01, -1.6809e-01,
          2.2704e-01, -3.4573e-01,  1.9350e-01, -8.6950e-02,  6.2820e-01,
          3.6054e-02, -3.0480e-01, -4.5010e-02, -2.9259e-01,  4.6916e-02,
          3.1867e-03,  1.3723e-01,  2.8357e-01,  4.4388e-03,  2.3751e-01,
         -1.5250e+00,  4.1601e-02, -2.4530e-01, -1.3552e-01,  7.8511e-02,
          2.6434e+00,  2.8209e-01, -1.0999e-01, -2.7832e-01, -1.0059e-01,
         -1.3455e-01,  1.8512e-01,  6.1003e-02, -7.2728e-02,  5.8264e-02,
         -5.3455e-01,  2.1291e-02,  6.1507e-01,  1.5536e-01,  4.4793e-01,
          1.4188e-02, -5.3610e-01,  1.3268e-01,  1.9627e-01, -2.2344e-01]),
 tensor([ 0.4802,  0.0527,  0.0179,  0.2030,  0.2559,  0.0311, -0.1485, -0.0743,
          0.1972, -0.2133,  0.0678, -0.0927, -0.0995, -0.1768,  0.4216,  0.1769,
         -0.3039, -0.1317,  0.0351, -0.0585,  0.0855,  0.0467,  0.3754, -0.1115,
          0.264

In [23]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Define the LSTM model architecture
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [24]:
gen_matrices = np.array(combined_df['GenMatrix'].values.tolist())


<ipython-input-24-5cadcfb371c8>:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  gen_matrices = np.array(combined_df['GenMatrix'].values.tolist())
<ipython-input-24-5cadcfb371c8>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gen_matrices = np.array(combined_df['GenMatrix'].values.tolist())


In [25]:
gen_matrices[0]

[tensor([-0.6388,  0.8221,  0.6981,  0.5711,  0.2418, -0.1767,  0.2791, -0.4686,
         -0.3980, -0.2200, -0.1481, -0.0502, -0.4037, -0.2298,  0.2376, -0.1930,
          0.0573, -0.4583, -0.6982,  0.0584, -0.0568, -0.4628,  0.5356,  0.0103,
         -0.2741, -0.5320,  0.0033, -0.2409,  0.1985,  0.0545,  2.5853,  0.3750,
          0.0605,  0.0932, -0.1876, -0.2977,  0.3662, -0.4391,  0.1142,  0.0547,
          0.2714, -0.1518,  0.2328, -0.1186, -0.1562,  0.2848,  0.0709,  0.4269,
          0.5056,  0.3155]),
 tensor([ 0.0607,  0.2446,  0.1136,  0.0501,  0.4626, -0.3692, -0.4194, -0.4245,
         -0.0517,  0.0305, -0.3850,  0.2517, -0.3255,  0.0917,  0.5974,  0.1086,
         -0.3259, -0.5328, -0.0833, -0.3223,  0.5537,  0.2215,  0.1738,  0.4101,
          0.4177, -1.3236, -0.4841,  0.0427,  0.0096, -0.3524,  2.7048, -0.1745,
         -0.0520, -0.4344, -0.2899, -0.3735, -0.0140, -0.1316, -0.1397, -0.2604,
          0.3387,  0.0951,  0.2089,  0.2302,  0.0066,  0.3785,  0.3102,  0.3359,

In [26]:
combined_df.head()

,Generation,label,GenMatrix
0,latest headlines on cnn business tl;dr the u.s...,ctrl,"[[tensor(-0.6388), tensor(0.8221), tensor(0.69..."
1,china wants to take a victory lap over its han...,ctrl,"[[tensor(0.5590), tensor(-0.1599), tensor(0.21..."
2,coronavirus disinformation creates challenges ...,ctrl,"[[tensor(0.5684), tensor(-0.1209), tensor(0.13..."
3,china coronavirus: eating wild animals made il...,ctrl,"[[tensor(0.5481), tensor(-0.2022), tensor(-0.1..."
4,china's economy could shrink for the first tim...,ctrl,"[[tensor(0.2362), tensor(0.1218), tensor(0.078..."


In [27]:
combined_df1 = combined_df[['GenMatrix','label']]

In [28]:
combined_df1['label'].unique()

array(['ctrl', 'fair', 'gpt', 'gpt3', 'gpt2', 'grover', 'human', 'pplm',
       'xlm', 'xlnet', 'instructgpt'], dtype=object)

In [29]:
ltoi = {l: i for i, l in enumerate(combined_df1['label'].unique())}
combined_df1['label1'] = combined_df1['label'].apply(lambda y: ltoi[y])

In [30]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence


# class GenMatrixDataset(Dataset):
#     def __init__(self, df):
#         self.labels = torch.tensor(df['label1'].values)
#         self.gen_matrices = torch.stack([torch.tensor(x) for x in df['GenMatrix'].values])

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         label = self.labels[idx]
#         gen_matrix = self.gen_matrices[idx]
#         return gen_matrix, label

class GenMatrixDataset(Dataset):
    def __init__(self, df):
        df = df[df['GenMatrix'].str.len() > 0]
        self.labels = torch.tensor(df['label1'].values)
        # self.labels=[]
        self.gen_matrices = []
        for x in df['GenMatrix'].values:
          if len(x) > 1:
            # print(torch.stack(x))
            self.gen_matrices.append(torch.stack(x))
          else:
            # print(x)
            if len(x)==1:
              self.gen_matrices.append(torch.stack(x))
            # print("Here")
        
        
        self.gen_matrices = pad_sequence(self.gen_matrices, batch_first=True)
        print(self.gen_matrices.size())
        print(len(self.labels))

        
        # self.gen_matrices = torch.stack(self.gen_matrices)
          
        # self.gen_matrices = torch.stack([torch.tensor(x[0]) if len(x) == 1 else torch.tensor(x) for x in df['GenMatrix'].values])
        # self.gen_matrices = torch.stack([torch.tensor(x) if len(x) > 1 else torch.tensor(x[0]).unsqueeze(0) for x in df['GenMatrix'].values])
        # self.gen_matrices = torch.stack([torch.tensor(x) if len(x) > 1 else torch.tensor(x).unsqueeze(0) for x in df['GenMatrix'].values])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.gen_matrices[idx], self.labels[idx]


# example usage:
# df = ... # your Pandas DataFrame with columns 'label' and 'GenMatrix'
dataset = GenMatrixDataset(combined_df1)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


torch.Size([11687, 161, 50])
11687


In [31]:
# class LSTM(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(LSTM, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):
#         h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device)
#         c0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device)
#         out, _ = self.lstm(x, (h0, c0))
#         out = self.fc(out[:, -1, :])
#         return out


In [32]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        super().__init__()
        #1. Initialize Embedding Layer
        # self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim, padding_idx = pad_idx)
        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = n_layers, dropout = dropout, bidirectional = bidirectional)
        #3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(hidden_dim * n_layers, output_dim)
        #4. Initialize Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        # embedding = self.embedding(text)
        embedding_dropout = self.dropout(text)
        #2. Run the LSTM along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions]; #hidden = [num layers * num directions, batch size, hid dim]
        output, (hidden,cell) = self.lstm(embedding_dropout)
        #3. Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        concat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(concat)     
        return self.fc(hidden)

In [33]:
# INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 161
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
# PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM( 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT)

In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 902,577 trainable parameters


In [35]:
# Split the dataset into train, test, and validation sets
total_len = len(dataset)
train_len = int(total_len * 0.8)
test_len = int(total_len * 0.1)
val_len = total_len - train_len - test_len

train_set, test_set, val_set = torch.utils.data.random_split(
    dataset, [train_len, test_len, val_len])

# Create data loaders for train, test, and validation sets
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)


In [36]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # get the index of the highest prediction for each instance
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [37]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # initialize the model and optimizer
# model = LSTM(input_dim=50, hidden_dim=64, output_dim=11).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # train the model
# num_epochs = 1
# criterion = nn.CrossEntropyLoss()
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     acc=[]
#     for i, (inputs, labels) in enumerate(train_loader):
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(inputs)
#         # print(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         acc.append(multiclass_accuracy(outputs,labels))
#         # running_loss += loss.item()
#         # _, predicted = torch.max(outputs.data, 1)
#         # total += labels.size(0)
#         # correct += (predicted == labels).sum().item()

#     epoch_loss = running_loss / len(dataloader)
#     epoch_acc = sum(acc) / len(acc)



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# initialize the model and optimizer
# model = LSTM(input_dim=50, hidden_dim=128, output_dim=11).to(device)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



# train the model
num_epochs = 20
criterion = nn.CrossEntropyLoss().to(device)
# criterion = nn.CrossEntropyLoss()



# move the model to the device
model.to(device)



# for epoch in range(num_epochs):
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     acc=[]
#     for i, (inputs, labels) in enumerate(train_loader):
#         inputs = inputs.to(device)
#         labels = labels.to(device)
#         inputs = torch.transpose(inputs, 0, 1)
#         # print(inputs.size())

#         optimizer.zero_grad()
#         outputs = model(inputs)
#         # print(outputs.size())
#         # print(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()

#         # acc = multiclass_accuracy(outputs,labels)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     epoch_loss = running_loss / len(train_loader.dataset)
#     epoch_acc = correct / total
#     print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
#           .format(epoch+1, num_epochs, epoch_loss, 100 * epoch_acc))


LSTM(
  (lstm): LSTM(50, 161, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=322, out_features=11, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [42]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# train the model
num_epochs = 50
criterion = nn.CrossEntropyLoss().to(device)
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    acc=[]
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        inputs = torch.transpose(inputs, 0, 1)
        # print(inputs.size())

        optimizer.zero_grad()
        outputs = model(inputs)
        # print(outputs.size())
        # print(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # acc = multiclass_accuracy(outputs,labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total


    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            inputs = torch.transpose(inputs, 0, 1)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = val_correct / val_total

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch+1, num_epochs, epoch_loss, 100 * epoch_acc))
    print('Epoch [{}/{}], Validation Loss: {:.4f}, Validation Accuracy: {:.2f}%'
          .format(epoch+1, num_epochs, val_loss, 100 * val_acc))

Epoch [1/50], Loss: 0.0352, Accuracy: 58.22%
Epoch [1/50], Validation Loss: 0.0325, Validation Accuracy: 62.39%
Epoch [2/50], Loss: 0.0350, Accuracy: 57.74%
Epoch [2/50], Validation Loss: 0.0332, Validation Accuracy: 61.45%
Epoch [3/50], Loss: 0.0344, Accuracy: 59.12%
Epoch [3/50], Validation Loss: 0.0317, Validation Accuracy: 62.14%
Epoch [4/50], Loss: 0.0342, Accuracy: 59.24%
Epoch [4/50], Validation Loss: 0.0315, Validation Accuracy: 63.76%
Epoch [5/50], Loss: 0.0337, Accuracy: 59.70%
Epoch [5/50], Validation Loss: 0.0329, Validation Accuracy: 61.45%
Epoch [6/50], Loss: 0.0337, Accuracy: 59.14%
Epoch [6/50], Validation Loss: 0.0313, Validation Accuracy: 64.53%
Epoch [7/50], Loss: 0.0331, Accuracy: 60.61%
Epoch [7/50], Validation Loss: 0.0320, Validation Accuracy: 62.82%
Epoch [8/50], Loss: 0.0328, Accuracy: 60.80%
Epoch [8/50], Validation Loss: 0.0307, Validation Accuracy: 65.04%
Epoch [9/50], Loss: 0.0325, Accuracy: 61.09%
Epoch [9/50], Validation Loss: 0.0308, Validation Accuracy:

In [43]:

for epoch in range(2):
    running_loss = 0.0
    correct = 0
    total = 0
    # acc=[]
    model.eval()
    with torch.no_grad():
      for i, (inputs, labels) in enumerate(test_loader):
          inputs = inputs.to(device)
          labels = labels.to(device)
          inputs = torch.transpose(inputs, 0, 1)
          # print(inputs.size())

          optimizer.zero_grad()
          outputs = model(inputs)
          # print(outputs.size())
          # print(inputs)
          loss = criterion(outputs, labels)
          # loss.backward()
          # optimizer.step()
          running_loss += loss.item()

          # acc = multiclass_accuracy(outputs,labels)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(test_loader.dataset)
    epoch_acc = correct / total
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch+1, num_epochs, epoch_loss, 100 * epoch_acc))




# def evaluate(model, iterator, criterion):
    
#     epoch_loss = 0
#     epoch_acc = 0
    
#     model.eval()
    
#     with torch.no_grad():
    
#         for batch in iterator:
            
#             text, text_lengths = batch.text
            
#             predictions = model(text, text_lengths).squeeze(1)
#             target = batch.label
#             target = target.type(torch.LongTensor)
            
#             loss = criterion(predictions.to(device), target.to(device))
            
#             acc = multiclass_accuracy(predictions, batch.label)

#             epoch_loss += loss.item()
#             epoch_acc += acc.item()



#     return epoch_loss / len(iterator), epoch_acc / len(iterator)

Epoch [1/50], Loss: 0.0565, Accuracy: 45.46%
Epoch [2/50], Loss: 0.0564, Accuracy: 45.46%


In [44]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/NLP/sentence_level_model.pt')


In [ ]:
# INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 161
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
# PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model1 = LSTM( 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT)

In [ ]:




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# initialize the model and optimizer
# model = LSTM(input_dim=50, hidden_dim=128, output_dim=11).to(device)
model1 = model1.to(device)

optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)



# train the model
num_epochs = 20
# criterion = nn.CrossEntropyLoss().to(device)
criterion = nn.NLLLoss().to(device)



# move the model to the device
model1.to(device)



for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    acc=[]
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        inputs = torch.transpose(inputs, 0, 1)
        # print(inputs.size())

        optimizer.zero_grad()
        outputs = model1(inputs)
        # print(outputs.size())
        # print(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # acc = multiclass_accuracy(outputs,labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch+1, num_epochs, epoch_loss, 100 * epoch_acc))
